In [41]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from scipy.stats import norm
import math as mt
import yfinance as yf

In [42]:
years = 15
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days=years*365)
 

In [43]:
tickers = ['^GSPC', '^DJI', '^IXIC', '^RUT', '^VIX', 'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'JPM', 'JNJ', 'V', 'PG', 'UNH', 'HD', 'DIS', 'MA']

In [44]:
adj_close_df = pd.DataFrame()
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    try:
        series = data['Adj Close']
    except KeyError:
        # fallback for different column layouts (MultiIndex or missing 'Adj Close')
        if isinstance(data.columns, pd.MultiIndex):
            if ('Adj Close', ticker) in data.columns:
                series = data[('Adj Close', ticker)]
            elif ('Close', ticker) in data.columns:
                series = data[('Close', ticker)]
            else:
                # try to extract the level by name
                try:
                    s = data.xs('Adj Close', axis=1, level=0)
                except Exception:
                    s = data.xs('Close', axis=1, level=0)
                # if xs returns a DataFrame, pick column for ticker if present, else take first column
                if isinstance(s, pd.DataFrame):
                    series = s[ticker] if ticker in s.columns else s.iloc[:, 0]
                else:
                    series = s
        else:
            if 'Close' in data.columns:
                series = data['Close']
            else:
                raise KeyError(f"'Adj Close' or 'Close' not found in downloaded data for {ticker}")
    adj_close_df[ticker] = series
print(adj_close_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

                  ^GSPC          ^DJI         ^IXIC         ^RUT       ^VIX  \
Date                                                                          
2011-01-14  1293.239990  11787.379883   2755.300049   807.570007  15.460000   
2011-01-18  1295.020020  11837.929688   2765.850098   807.559998  15.870000   
2011-01-19  1281.920044  11825.290039   2725.360107   786.890015  17.309999   
2011-01-20  1280.260010  11822.799805   2704.290039   778.080017  17.990000   
2011-01-21  1283.349976  11871.839844   2689.540039   773.179993  18.469999   
...                 ...           ...           ...          ...        ...   
2026-01-05  6902.049805  48977.179688  23395.820312  2547.919922  14.900000   
2026-01-06  6944.819824  49462.078125  23547.169922  2582.899902  14.750000   
2026-01-07  6920.930176  48996.078125  23584.279297  2575.419922  15.380000   
2026-01-08  6921.459961  49266.109375  23480.019531  2603.909912  15.450000   
2026-01-09  6966.279785  49504.070312  23671.349609 

In [45]:
log_returns = np.log(adj_close_df / adj_close_df.shift(1)).dropna()
print(log_returns)

               ^GSPC      ^DJI     ^IXIC      ^RUT      ^VIX      AAPL  \
Date                                                                     
2012-05-21  0.015909  0.010863  0.024324  0.023059 -0.131371  0.056626   
2012-05-22  0.000486 -0.000134 -0.002859 -0.006574  0.021129 -0.007708   
2012-05-23  0.001692 -0.000533  0.003881  0.007267 -0.006695  0.024107   
2012-05-24  0.001379  0.002685 -0.003775  0.001828 -0.036019 -0.009227   
2012-05-25 -0.002168 -0.005997 -0.000652 -0.000209  0.010162 -0.005374   
...              ...       ...       ...       ...       ...       ...   
2026-01-05  0.006334  0.012219  0.006870  0.015704  0.026523 -0.013934   
2026-01-06  0.006178  0.009852  0.006448  0.013635 -0.010118 -0.018504   
2026-01-07 -0.003446 -0.009466  0.001575 -0.002900  0.041825 -0.007768   
2026-01-08  0.000077  0.005496 -0.004431  0.011002  0.004541 -0.004967   
2026-01-09  0.006455  0.004818  0.008116  0.007770 -0.064150  0.001273   

                MSFT     GOOGL      A

In [46]:
portfolio = 10000000
weight = np.array([1/len(tickers)]*len(tickers))
print(weight)

[0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.05 0.05 0.05 0.05 0.05 0.05]


In [47]:
historical_returns = (log_returns*weight).sum(axis=1)
print(historical_returns)

Date
2012-05-21    0.004825
2012-05-22    0.000837
2012-05-23    0.005011
2012-05-24   -0.001480
2012-05-25   -0.005512
                ...   
2026-01-05    0.009442
2026-01-06    0.003861
2026-01-07   -0.001012
2026-01-08    0.004284
2026-01-09    0.001984
Length: 3430, dtype: float64


In [50]:
Days = 250
rang_returns = historical_returns.rolling(window=Days).sum().dropna()
print(rang_returns)

Date
2013-05-20    0.254824
2013-05-21    0.252383
2013-05-22    0.245305
2013-05-23    0.240978
2013-05-24    0.245459
                ...   
2026-01-05    0.103058
2026-01-06    0.102604
2026-01-07    0.106627
2026-01-08    0.111728
2026-01-09    0.120787
Length: 3181, dtype: float64


In [49]:
confindence_level = 0.95
Value_at_Risk = -np.percentile(rang_returns, (1 - confindence_level) * 100) * portfolio
print(f"Value at Risk (VaR) at {confindence_level*100}% confidence level over {Days} days: ${Value_at_Risk:,.2f}")

Value at Risk (VaR) at 95.0% confidence level over 252 days: $1,252,485.80
